<a href="https://colab.research.google.com/github/zahraDehghanian97/sharpening_filtering/blob/master/BioMedical_Image_Analysis_Hw2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Please insert your full name:** Zahra Dehghanian

**And your student number:** 401300417



---



# Q1 (Sharpening)


## Import dependencies:

## Read the image and use four different methods to sharpen the image. Briefly discuss your main approach in each of the methods.

You are allowed to use any kind of different methods. The recommanded ones are as follow:

1) unsharp mask filter

2) second deriative or guassian laplacian

3) unsharp mask in frequency domain

4) laplacian in frequency domain , sharpening with laplacian

## First Method

## Second Method

## Third Method

## Forth Method

### Compare different methods quality and time.

# Q2(Filtering)

## Import dependencies:

## Section 1

#### Sobel

#### gaussian

#### laplacian

## Section 2

## Compare and contrast results